# Function Calling with local model using mistral-inference

Function calling allows Mistral models to connect to external tools. By integrating Mistral models with external tools such as user defined functions or APIs, users can easily build applications catering to specific use cases and practical problems. In this guide, for instance, we wrote two functions for tracking payment status and payment date. We can use these two tools to provide answers for payment-related queries.

At a glance, there are four steps with function calling:

- User: specify tools and query
- Model: Generate function arguments if applicable
- User: Execute function to obtain tool results
- Model: Generate final answer

In this guide, we will walk through a simple example to demonstrate how function calling works with Mistral models in these four steps.

Before we get started, let’s assume we have a dataframe consisting of payment transactions. When users ask questions about this dataframe, they can use certain tools to answer questions about this data. This is just an example to emulate an external database that the LLM cannot directly access.

In [ ]:
!pip install pandas mistral-inference

In [8]:
import pandas as pd

# Assuming we have the following data
data = {
    'transaction_id': ['T1001', 'T1002', 'T1003', 'T1004', 'T1005'],
    'customer_id': ['C001', 'C002', 'C003', 'C002', 'C001'],
    'payment_amount': [125.50, 89.99, 120.00, 54.30, 210.20],
    'payment_date': ['2021-10-05', '2021-10-06', '2021-10-07', '2021-10-05', '2021-10-08'],
    'payment_status': ['Paid', 'Unpaid', 'Paid', 'Paid', 'Pending']
}

# Create DataFrame
df = pd.DataFrame(data)

## Step 1. User: specify tools and query

### Tools

Users can define all the necessary tools for their use cases.

- In many cases, we might have multiple tools at our disposal. For example, let’s consider we have two functions as our two tools: `retrieve_payment_status` and `retrieve_payment_date` to retrieve payment status and payment date given transaction ID.

In [9]:
def retrieve_payment_status(df: data, transaction_id: str) -> str:
    if transaction_id in df.transaction_id.values: 
        return json.dumps({'status': df[df.transaction_id == transaction_id].payment_status.item()})
    return json.dumps({'error': 'transaction id not found.'})

def retrieve_payment_date(df: data, transaction_id: str) -> str:
    if transaction_id in df.transaction_id.values: 
        return json.dumps({'date': df[df.transaction_id == transaction_id].payment_date.item()})
    return json.dumps({'error': 'transaction id not found.'})



- In order for Mistral models to understand the functions, we need to outline the function specifications with a JSON schema. Specifically, we need to describe the type, function name, function description, function parameters, and the required parameter for the function. Since we have two functions here, let’s list two function specifications in a list.

In [1]:
from mistral_common.protocol.instruct.tool_calls import Function, Tool

tools=[
        Tool(
            function=Function(
                name="retrieve_payment_status",
                description="Get payment status of a transaction",
                parameters={
                    "type": "object",
                    "properties": {
                        "transaction_id": {
                        "type": "string",
                        "description": "The transaction id.",
                        }
                    },
                    "required": ["transaction_id"],
                },
            )
        ),
        Tool(
            function=Function(
                name="retrieve_payment_date",
                description="Get payment date of a transaction",
                parameters={
                    "type": "object",
                    "properties": {
                        "transaction_id": {
                        "type": "string",
                        "description": "The transaction id.",
                        }
                    },
                    "required": ["transaction_id"],
                },
            )
        )
    ]



- Then we organize the two functions into a dictionary where keys represent the function name, and values are the function with the df defined. This allows us to call each function based on its function name.

In [10]:
import functools

names_to_functions = {
  'retrieve_payment_status': functools.partial(retrieve_payment_status, df=df),
  'retrieve_payment_date': functools.partial(retrieve_payment_date, df=df)
}


### User query

Suppose a user asks the following question: “What’s the status of my transaction?” A standalone LLM would not be able to answer this question, as it needs to query the business logic backend to access the necessary data. But what if we have an exact tool we can use to answer this question? We could potentially provide an answer!

In [ ]:
from mistral_common.protocol.instruct.messages import UserMessage

messages=[UserMessage(content="What's the status of my transaction T1003?")
        ]

## Step 2. Model: Generate function arguments 

How do Mistral models know about these functions and know which function to use? We provide both the user query and the tools specifications to Mistral models. The goal in this step is not for the Mistral model to run the function directly. It’s to 1) determine the appropriate function to use , 2) identify if there is any essential information missing for a function, and 3) generate necessary arguments for the chosen function.

In [ ]:
from mistral_inference.model import Transformer
from mistral_inference.generate import generate

from mistral_common.tokens.tokenizers.mistral import MistralTokenizer

from mistral_common.protocol.instruct.request import ChatCompletionRequest

- We specify the location to load the mistral model and tokenizer.  These models are downloaded from [mistral-inference](https://github.com/mistralai/mistral-inference) repo. We have downloaded the mistral 7BInstructv3 model

In [2]:
tokenizer = MistralTokenizer.from_file("./mistral_models/7B_instruct/tokenizer.model.v3")  # change to extracted tok>
model = Transformer.from_folder("./mistral_models/7B_instruct")  # change to extracted model dir

In [3]:
completion_request = ChatCompletionRequest(tools, messages,)


In [4]:
tokens = tokenizer.encode_chat_completion(completion_request).tokens


max_tokens control the output of model as tokens, set it based on required function call.

In [5]:
out_tokens, _ = generate([tokens], model, max_tokens=30, temperature=0.0, eos_id=tokenizer.instruct_tokenizer.tokenizer.eos_id)

In [6]:
result = tokenizer.instruct_tokenizer.tokenizer.decode(out_tokens[0])
#remove any blank space from token output
result = result.strip()
result

## Step 3. User: Execute function to obtain tool results

How do we execute the function? Currently, it is the user’s responsibility to execute these functions and the function execution lies on the user side. In the future, we may introduce some helpful functions that can be executed server-side.

Let’s extract some useful function information from model response including function_name and function_params. It’s clear here that our Mistral model has chosen to use the function `retrieve_payment_status` with the parameter `transaction_id` set to T1001.

In [11]:
import json
tool_call = json.loads(result)
function_name = tool_call[0]["name"]
function_params = (tool_call[0]["arguments"]) 
print("\nfunction_name: ", function_name, "\nfunction_params: ", function_params)


function_name:  retrieve_payment_status 
function_params:  {'transaction_id': 'T1003'}


In [12]:
function_result = names_to_functions[function_name](**function_params)
function_result

'{"status": "Paid"}'